# Natural Computing Project - BiomakerCA - Simulating Plant Reproducibility 

### 1. Overrides

In [4]:
# Overriding the default environment logic with a custom one
import overrides.env_logic_override as env_override
import self_organising_systems.biomakerca.env_logic as env_logic
env_logic.process_energy = env_override.process_energy


# Overriding the default step maker with a custom one
import overrides.step_maker_override as step_maker_override
import self_organising_systems.biomakerca.step_maker as step_maker
step_maker.step_env = step_maker_override.step_env



### 2. Imports

In [2]:
import jax.random as jr
import mediapy as media
from IPython.display import Video
from jax import vmap
from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import (
    BasicMutator, RandomlyAdaptiveMutator)

from biomaker_utils import perform_evaluation, perform_simulation, start_simulation
from configs.seasons_config import SeasonsConfig

import numpy as np


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/jax/_src/api_util.py:231: SyntaxWarning: Jitted function has invalid argnames {'config', 'mutator', 'n_steps', 'n_max_programs'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


In [4]:
#@title imports & notebook utilities
from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import BasicMutator
from self_organising_systems.biomakerca.mutators import RandomlyAdaptiveMutator
from self_organising_systems.biomakerca.step_maker import step_env
from self_organising_systems.biomakerca.display_utils import zoom
from self_organising_systems.biomakerca.custom_ipython_display import display
from self_organising_systems.biomakerca.env_logic import ReproduceOp
from self_organising_systems.biomakerca.env_logic import env_perform_one_reproduce_op

import cv2
import numpy as np
import jax.random as jr
import jax.numpy as jp
from jax import vmap
from jax import jit
import jax
import time

import tqdm
import mediapy as media
from functools import partial

In [5]:
def pad_text(img, text):
  font = cv2.FONT_HERSHEY_SIMPLEX
  orgin = (5, 15)
  fontScale = 0.5
  color = (0, 0, 0)
  thickness = 1

  # ensure to preserve even size (assumes the input size was even.
  new_h = img.shape[0]//15
  new_h = new_h if new_h % 2 == 0  else new_h + 1
  img = np.concatenate([np.ones([new_h, img.shape[1], img.shape[2]]), img], 0)
  img = cv2.putText(img, text, orgin, font, fontScale, color, thickness, cv2.LINE_AA)
  return img

In [6]:
ec_id = "pestilence" #@param ['persistence', 'pestilence', 'collaboration', 'sideways']
env_width_type = "square" #@param ['wide', 'landscape', 'square', 'petri']
soil_unbalance_limit = 0 #@param [0, "1/3"] {type:"raw"}

env_and_config = evm.get_env_and_config(ec_id, width_type=env_width_type)
st_env, config = env_and_config
config.soil_unbalance_limit = soil_unbalance_limit

agent_model = "minimal" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "basic" #@param ['basic', 'randomly_adaptive']
sd = 1e-2 if mutator_type == "basic" and agent_model == "basic" else 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

BasicAgentLogic.dsm_num_params = 0
BasicAgentLogic.nsl_num_params = 176
BasicAgentLogic.denm_num_params = 80
BasicAgentLogic.excl_num_params = 41
BasicAgentLogic.repr_num_params = 2
BasicAgentLogic.num_params = 299


In [7]:
base_config = SeasonsConfig()

st_env, env_config = evm.get_env_and_config(base_config.ec_id, width_type=base_config.env_width_type)
env_config.soil_unbalance_limit = base_config.soil_unbalance_limit
env_config.nutrient_cap = np.asarray([20, 20])
env_config.specialize_cost = np.asarray([0.030, 0.030])

agent_logic = BasicAgentLogic(env_config, minimal_net=base_config.agent_model == "minimal")
env_config.max_lifetime = 2000

sd = 1e-2 if base_config.mutator_type == "basic" and base_config.agent_model == "basic" else 1e-3
mutator = (
    BasicMutator(sd=sd, change_perc=0.2)
    if base_config.mutator_type == "basic"
    else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)
)

print("\n\nCurrent config:")
print("\n".join("%s: %s" % item for item in vars(env_config).items()))

ku, key = jr.split(base_config.key)
programs = vmap(agent_logic.initialize)(jr.split(ku, base_config.n_max_programs))
programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)

env = st_env

file_version = "v3"

print(base_config.out_file)

In [6]:
### 4. Performing Basic Simulation

TypeError: perform_simulation() missing 1 required positional argument: 'video'

In [4]:
### 4. Performing Basic Simulation

In [5]:
# spring_agent_logic = agent_logic

# frame = start_simulation(env, base_config, env_config)
# with media.VideoWriter(
# 	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
# ) as video:
# 	perform_simulation(
# 		env, programs, base_config, env_config, spring_agent_logic, mutator, key, video, frame
# 	)

# Video(base_config.out_file)

In [6]:
# perform_evaluation(env, programs, st_env, env_config, agent_logic, mutator, base_config)

### 5. Simulating Seasons

In [7]:
APPEND_ALL_SEASONS_IN_ONE_VIDEO = True

#### Seasons!

In [8]:
## Examples for modifying the config
## Uncomment relevant lines or do like them.

## Regardless, to trigger the recomputation of step_env and similar,
## config needs to be a new object! So, first, we create a new copy.
import copy
config = copy.copy(config)

## Change simple isolated parameters (most of them)
# config.struct_integrity_cap = 100
# config.max_lifetime = 500
## Vectors can be modified either by writing new vectors:
# config.dissipation_per_step = jp.array([0.02, 0.02])
## Or by multiplying previous values. Note that they are immutable!
# config.dissipation_per_step = config.dissipation_per_step * 2

## agent_state_size is trickier, because it influences env_state_size.
## So you can either create a new config:
## Note that you would have to insert all values that you don't want to take
## default initializations.
# config = evm.EnvConfig(agent_state_size=4)
## Or you can just modify env_state_size as well.
## (env_state_size = agent_state_size + 4) for now.
# config.agent_state_size = 4
# config.env_state_size = config.agent_state_size + 4


In [9]:
if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	base_config.out_file = f"spring{file_version}.mp4"

frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	# Spring 1
	base_config.AIR_DIFFUSION_RATE = 0.09
	base_config.SOIL_DIFFUSION_RATE = 0.09

	step, env, programs = perform_simulation(
		env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, season="Spring 1"
	)

	# # Summer 1
	# base_config.AIR_DIFFUSION_RATE = 0.1
	# base_config.SOIL_DIFFUSION_RATE = 0.1

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Summer 1"
	# )

	# # Autumn 1
	# base_config.AIR_DIFFUSION_RATE = 0.04
	# base_config.SOIL_DIFFUSION_RATE = 0.04

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Autumn 1"
	# )

	# # Winter 1
	# base_config.AIR_DIFFUSION_RATE = 0.01
	# base_config.SOIL_DIFFUSION_RATE = 0.01

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Winter 1"
	# )

	# # Spring 2
	# base_config.AIR_DIFFUSION_RATE = 0.09
	# base_config.SOIL_DIFFUSION_RATE = 0.09

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Spring 2"
	# )

	# # Summer 2
	# base_config.AIR_DIFFUSION_RATE = 0.1
	# base_config.SOIL_DIFFUSION_RATE = 0.1

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Summer 2"
	# )

	# # Autumn 2
	# base_config.AIR_DIFFUSION_RATE = 0.04
	# base_config.SOIL_DIFFUSION_RATE = 0.04

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Autumn 2"
	# )

	# # Winter 2
	# base_config.AIR_DIFFUSION_RATE = 0.01
	# base_config.SOIL_DIFFUSION_RATE = 0.01

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Winter 2"
	# )


# if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
Video(base_config.out_file)

Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>


  0%|          | 0/125 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int32 to dtype=uint32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
2024-04-30 14:40:03.894055: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.003301s
Constant folding an instruction is taking > 1s:

  %reduce.85 = f32[72,128,9,6]{3,2,1,0} reduce(f32[1,72,128,9,6]{4,3,2,1,0} %broadcast.30, f32[] %constant.155), dimensions={0}, to_apply=%region_125.7136, metadata={op_name="jit(step_env)/jit(main)/reduce_sum[axes=(0,)]" source_file="/Users/laurastritzel/Desktop/Radboud/semester 2/Natural Computing/final project/NaCO_project/overrides/step_maker_override.py" source_line=179}

This isn't necessarily a bug; constant-folding is inherently a trade-off between comp

In [18]:
agentTypes = env.state_grid

# Count of each type of agent
zeros = np.count_nonzero(agentTypes == 0) # unspecialized
ones = np.count_nonzero(agentTypes == 1) # root
twos = np.count_nonzero(agentTypes == 2) # leaf
threes = np.count_nonzero(agentTypes == 3) # flower

print("Count of unspecialized in grid: ", zeros)
print("Count of roots in grid: ", ones)
print("Count of leafs in grid: ", twos)
print("Count of flowers in grid: ", threes)


Count of unspecialized in grid:  38842
Count of roots in grid:  0
Count of leafs in grid:  0
Count of flowers in grid:  0


In [11]:
perform_evaluation(env, programs, st_env, env_config, agent_logic, mutator, base_config)

Extracted 3 programs.
Took 18.060476779937744 seconds
Total number of agents [14091] 14091.0 0.0
Extinction events [0] 0.0 0.0
Number of flowers:  0
Count of unspecialized in grid:  8579
Count of roots in grid:  322
Count of leafs in grid:  315
Count of flowers in grid:  0


In [16]:
if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	base_config.out_file = f"spring{file_version}.mp4"

frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	# Spring 1
	base_config.AIR_DIFFUSION_RATE = 0.07
	base_config.SOIL_DIFFUSION_RATE = 0.07

	step, env, programs = perform_simulation(
		env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, season="Spring 1"
	)

	# Summer 1
	base_config.AIR_DIFFUSION_RATE = 0.1
	base_config.SOIL_DIFFUSION_RATE = 0.1

	step, env, programs = perform_simulation(
		env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Summer 1"
	)

	# Autumn 1
	base_config.AIR_DIFFUSION_RATE = 0.04
	base_config.SOIL_DIFFUSION_RATE = 0.04

	step, env, programs = perform_simulation(
		env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Autumn 1"
	)

	# # Winter 1
	# base_config.AIR_DIFFUSION_RATE = 0.01
	# base_config.SOIL_DIFFUSION_RATE = 0.01

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Winter 1"
	# )

	# # Spring 2
	# base_config.AIR_DIFFUSION_RATE = 0.09
	# base_config.SOIL_DIFFUSION_RATE = 0.09

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Spring 2"
	# )

	# # Summer 2
	# base_config.AIR_DIFFUSION_RATE = 0.1
	# base_config.SOIL_DIFFUSION_RATE = 0.1

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Summer 2"
	# )

	# # Autumn 2
	# base_config.AIR_DIFFUSION_RATE = 0.04
	# base_config.SOIL_DIFFUSION_RATE = 0.04

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Autumn 2"
	# )

	# # Winter 2
	# base_config.AIR_DIFFUSION_RATE = 0.01
	# base_config.SOIL_DIFFUSION_RATE = 0.01

	# step, env, programs = perform_simulation(
	# 	env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Winter 2"
	# )

# if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
Video(base_config.out_file)

100%|██████████| 125/125 [00:25<00:00,  4.96it/s]


In [17]:
perform_evaluation(env, programs, st_env, env_config, agent_logic, mutator, base_config)

Extracted 3 programs.
Took 18.52941083908081 seconds
Total number of agents [14091] 14091.0 0.0
Extinction events [0] 0.0 0.0
Number of flowers:  0
Count of unspecialized in grid:  7635
Count of roots in grid:  835
Count of leafs in grid:  746
Count of flowers in grid:  0


In [10]:
from IPython.display import Video

Video(out_file)

In [11]:
def count_agents_f(env, etd):
  return etd.is_agent_fn(env.type_grid).sum()

@partial(jit, static_argnames=["config", "agent_logic", "mutator", "n_steps", "n_max_programs"])
def evaluate_biome(key, st_env, config, agent_logic, mutator, n_steps,
                   init_program=None, n_max_programs=128):
  def body_f(i, carry):
    key, env, programs, tot_agents_n = carry
    ku, key = jr.split(key)

    env, programs = step_env(
        ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=True, mutator=mutator)

    tot_agents_n += count_agents_f(env, config.etd)
    return key, env, programs, tot_agents_n

  if init_program is None:
    ku, key = jr.split(key)
    programs = vmap(agent_logic.initialize)(jr.split(ku, n_max_programs))
    ku, key = jr.split(key)
    programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)
  else:
    programs = jp.repeat(init_program[None, :], n_max_programs, axis=0)

  key, env, programs, tot_agents_n = jax.lax.fori_loop(
      0, n_steps, body_f, (key, st_env, programs, 0))

  # check whether they got extinct:
  is_extinct = (count_agents_f(env, config.etd) == 0).astype(jp.int32)
  return tot_agents_n, is_extinct

In [12]:
what_to_evaluate = "initialization" # @param ["initialization", "extracted"]

key = jr.PRNGKey(123)

n_steps = 100
n_reps = 3

if what_to_evaluate == "initialization":
  init_programs = None
else:
  # Extract a living program from the final environment.
  aid_flat = env.agent_id_grid.flatten()
  is_agent_flat = evm.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
  n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=N_MAX_PROGRAMS)
  alive_programs = programs[n_alive_per_id>0]
  print("Extracted {} programs.".format(alive_programs.shape[0]))
  assert alive_programs.shape[0] >= n_reps, "Not enough alive programs found."

  init_programs = alive_programs[:n_reps]


t_st = time.time()
key, ku = jr.split(key)
b_tot_agents_n, b_is_extinct = jit(vmap(partial(
    evaluate_biome, st_env=st_env, config=config, agent_logic=agent_logic,
    mutator=mutator, n_steps=n_steps)))(jr.split(ku, n_reps), init_program=init_programs)
print("Took", time.time()-t_st, "seconds")
print("Total number of agents", b_tot_agents_n, b_tot_agents_n.mean(), b_tot_agents_n.std())
print("Extinction events", b_is_extinct, b_is_extinct.mean(), b_is_extinct.std())

/home/janneke/anaconda3/envs/NaCo3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int32 to dtype=uint32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
2024-04-19 16:01:25.475131: E external/xla/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  %reduce.133 = f32[3,72,72,9,6]{4,3,2,1,0} reduce(f32[3,1,72,72,9,6]{5,4,3,2,1,0} %broadcast.1479, f32[] %constant.387), dimensions={1}, to_apply=%region_130.5960, metadata={op_name="jit(<unnamed function>)/jit(main)/vmap(jit(evaluate_biome))/while/body/jit(step_env)/reduce_sum[axes=(1,)]" source_file="/tmp/ipykernel_4692/1170826805.py" source_line=11}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to keep constant folding from taking 

Took 63.143229722976685 seconds
Total number of agents [19166 11467  2939] 11190.667 6627.526
Extinction events [0 0 0] 0.0 0.0
